In [1]:

from dask_jobqueue import SLURMCluster # pip 
from dask.distributed import Client, progress 
import dask


import matplotlib as mpl
# Define Agg as Backend for matplotlib when no X server is running
mpl.use('Agg')
import socket
import os

In [2]:

def squeue_user(username = "$USER"):
    _squeue_user = os.system("squeue --user="+str(username))
    return _squeue_user 

with os.popen("pwd ") as f:
    _pwd = f.readline()

In [3]:
cluster = SLURMCluster(
    name='dask-cluster', 
    cores=256,    
    memory="500 GB", 
    project="bb1153",
    queue= "compute", 
    #interface='ib0',
    walltime='04:30:50',
)
client = Client(cluster)
print(cluster.job_script())

/home/b/b382267/mambaforge/envs/aqua/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=256
#SBATCH --mem=466G
#SBATCH -t 04:30:50

/home/b/b382267/mambaforge/envs/aqua/bin/python -m distributed.cli.dask_worker tcp://136.172.124.3:44499 --nthreads 16 --nworkers 16 --memory-limit 29.10GiB --name dummy-name --nanny --death-timeout 60



/home/b/b382267/mambaforge/envs/aqua/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33757 instead
  warnings.warn(
/home/b/b382267/mambaforge/envs/aqua/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


In [ ]:
cluster.scale(jobs=1)



In [4]:
squeue_user()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4160211   compute dask-wor  b382267  R      33:06      1 l40065


0

In [5]:


import sys
if 'tr_pr_mod' not in sys.modules:
    import diagnostics.tropical_rainfall.tr_pr_mod
else:                      
    import importlib      
    importlib.reload(diagnostics.tropical_rainfall.tr_pr_mod) # reload module on subsequent runs
    
from  diagnostics.tropical_rainfall.tr_pr_mod import TR_PR_Diagnostic




In [6]:
diag = TR_PR_Diagnostic()

diag.num_of_bins = 15
diag.first_edge = 0
diag.width_of_bin = 2000*10**(-6)/diag.num_of_bins

last_edge = diag.first_edge  + diag.num_of_bins*diag.width_of_bin

In [7]:
from aqua import Reader
from aqua.reader import catalogue

In [8]:
catalogue()

IFS	tco3999-ng5	2.8km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- interpolated_sp_ci	
IFS	tco2559-ng5	4km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
IFS	tco1279-orca025	9km baseline, coupled to NEMO, deep conv ON
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	

FESOM	tco3999-ng5	2.5km experiment, coupuled with IFS
	- elem_grid	
	- node_grid	
	- np	nearest-neighbor interpolation to lat-lon grid
	- interpolated_global2d	
	- interpolated_global_TS	
	- interpolated_global_UV	
	- interpolated_np	
	- interpolated_sp	
	- original_2d	original 2d output
	- original_3d	original 3d output
FESOM	tco2559-ng5	5km experiment, coupuled with IFS
	- elem_grid	
	- node_grid	
	- interpolated_global2d	
	- interpolated_global_TS	
	- interpolated_global_UV	
	- interpolated_np	
	- interpolated_sp	
	- original_2d	orig

config:
  args:
    path: ./config/catalog.yaml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


In [ ]:
reader = Reader(model="ICON", exp="ngc2009", source="atm_2d_ml_R02B06")
#data = reader.retrieve(fix=False)
#data = reader.retrieve(apply_unit_fix=False)
ICON_2009 = reader.retrieve()

In [ ]:
ICON_2009_small = ICON_2009['pr'][0:10,:]
ICON_2009_small = ICON_2009_small.compute()

In [9]:
import xarray as xr 
import matplotlib.pyplot as plt
import numpy as np 

In [ ]:
diag.time_interpreter_b(ICON_2009)

diag.time_interpreter_b(ICON_2009_small) 

## Id regrided then we can

### Animation

In [ ]:
diag.animated(DS,  label = ' ')

In [ ]:

scale_factor = 90/diag.trop_lat
fig = plt.figure(figsize=(8*scale_factor,6))
diag.ds_per_lat_range(DS)['pr'][0,:,:].plot(vmin=0, vmax=100)


In [ ]:

plt.plot(diag.mean_per_timestep(DS), label= 'mean')
plt.plot(diag.median_per_timestep(DS), label='median')

plt.xlabel('Time', fontsize=12)
plt.ylabel('Precipitation per day', fontsize=12)
plt.title('Mean/median values of precipitation', fontsize =16)
plt.legend(fontsize=12)

#plt.yscale('log')
print('gmean ....')


plt.savefig("./figures/mean_and_median_trop_pr.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')